# Predict

In [9]:
def generate_input(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    obs = [np.zeros([4, 8, 2]), np.zeros([4])]
    
    for card in hand_cards:
        obs[0][card.color.value][card.value.value][0] = 1
        
    if table_card is not None:
        obs[0][table_card.color.value][table_card.value.value][1] = 1

    obs[1][0] = (own_tricks == 1 or own_tricks == 3)
    obs[1][1] = (own_tricks == 2 or own_tricks == 3)

    obs[1][2] = (opponent_tricks == 1 or opponent_tricks == 3)
    obs[1][3] = (opponent_tricks == 2 or opponent_tricks == 3)
    
    return obs

In [10]:
def card(color, value):
    for card in env.cards:
        if card.value == value and card.color == color:
            return card
        
    return None

In [276]:
def calc_correct_output_sample(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    global sample_outputs, my_state
    env.reset()
    env.cards_left = env.cards[:]
    random.shuffle(env.cards_left)
    
    for card in hand_cards:
        env.cards_left.remove(card)
    env.players[0].hand_cards = hand_cards[:]
        
    env.players[1].hand_cards = []
    for i in range(len(hand_cards) - (1 if table_card is not None else 0)):
        env.players[1].hand_cards.append(env.cards_left.pop())      
    
    env.players[0].tricks = own_tricks
    env.players[1].tricks = opponent_tricks
    env.table_card = table_card
    
    my_state = env.get_state()
    
    root = State(1, env.get_state(), env.current_player)
    a, p = mcts_game_step(root, 100)
    env.set_state(root.env_state)
    print(p)
    #draw_tree(root)
    #print("")
    #draw_path(root)
    #obs, rew, is_done, _ = env.step(hand_cards[0].id)
   # print(rew, is_done)

    #env.render('human')
    #root = State(1, env.get_state())
    #a, p = mcts_game_step(root, 1000, False)
    #draw_tree(root)
    #print("")
    #draw_tree(root.childs[0])
    
    #draw_path(root)
    #reset_samples(1)
    p = search(env.regenerate_obs())[1]
    #postprocess_samples()
        
    return p#sample_outputs[0]

In [12]:
def calc_correct_output(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):    
    correct_output = None
    for i in range(100):
        sample_outputs = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks)
        if correct_output is None:
            correct_output = sample_outputs
        else:
            correct_output += sample_outputs
    return correct_output / 100

In [13]:
def predict(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
    output = model.predict_single(input)[0]
    correct_output = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks)
    print(output,correct_output)
    id = 0
    for card in hand_cards:
        print(str(card.color) + " " + str(card.value) + " => " + str(output[card.id]) + " (" + str(correct_output[id]) + ")")
        id += 1

In [339]:
def create_eval_cache():
    eval_cache_input = []
    eval_cache_output = []
    for i in range(1000):
        own_tricks = random.randint(0, 2)
        opponent_tricks = random.randint(0, 2)
        table_card_enabled = random.randint(0, 1)
        hand_cards = random.sample(env.cards, (5 - own_tricks - opponent_tricks) * 2 + table_card_enabled)
        table_card = hand_cards.pop() if table_card_enabled else None
        
        opponent_hand_cards = []
        for i in range(len(hand_cards) // 2):
            opponent_hand_cards.append(hand_cards.pop())

        eval_cache_input.append(generate_input(hand_cards, table_card, own_tricks, opponent_tricks, opponent_hand_cards))
        output = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks, opponent_hand_cards)
        
        outputs_per_card = np.zeros((32,))
        for i in range(len(hand_cards)):
            outputs_per_card[hand_cards[i].id] = output[i]
            
        eval_cache_output.append(outputs_per_card)
        
    return eval_cache_input, eval_cache_output
        
def eval(model, eval_cache_input, eval_cache_output):
    correct = 0
    for i in range(len(eval_cache_input)):

        output = model.predict_single(eval_cache_input[i])[0]
        correct_output = eval_cache_output[i]
                    
        correct += correct_output[np.argmax(output)] == correct_output[np.argmax(correct_output)]
        #if correct_output[np.argmax(output)] != correct_output[np.argmax(correct_output)]:
        #    print(i, correct_output[np.argmax(output)], correct_output[np.argmax(correct_output)])
            
    return correct / len(eval_cache_input)


In [42]:
eval_cache_input, eval_cache_output = create_eval_cache()

In [18]:
with open('eval_cache', 'wb') as fp:
    pickle.dump([eval_cache_input, eval_cache_output], fp)

In [92]:
with open ('eval_cache', 'rb') as fp:
    eval_cache_input, eval_cache_output = pickle.load(fp)

In [122]:
eval(model, eval_cache_input, eval_cache_output)

8 0.05 0.07
9 0.65 0.77
11 0.9 0.98
14 0.21 0.39
24 0.4 0.58
27 0.37 0.4
28 0.17 0.44
33 0.21 0.33
37 0.79 0.92
40 0.33 0.44
50 0.05 0.12
54 0.17 0.38
62 0.19 0.33
66 0.1 0.13
75 0.3 0.55
81 0.7 0.71
87 0.8 0.9
88 0.92 0.93
95 0.76 0.87
98 0.61 0.83


0.8

In [128]:
eval_cache_input[50]

[array([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [1., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [1., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 1.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [1., 0.],
         [0., 0.]]]), array([1., 0., 1., 0.])]

In [168]:
val = []
for i in range(5):
    val.append(calc_correct_output_sample([
            card(Color.EICHEL, Value.ZEHN), 
            card(Color.GRUEN, Value.KOENIG),
            card(Color.SCHELLN, Value.KOENIG)
        ], card(Color.HERZ, Value.UNTER), 1, 1))
np.mean(val, 0)


array([0.41414141, 0.21818182, 0.36767677])

In [171]:
env.set_state(my_state)
env.render('human')

True

In [161]:
calc_correct_output([
            card(Color.EICHEL, Value.ZEHN), 
            card(Color.GRUEN, Value.KOENIG),
            card(Color.SCHELLN, Value.KOENIG)
        ], card(Color.HERZ, Value.UNTER), 1, 1)

array([0.42666667, 0.29717172, 0.27616162])

In [118]:
predict([
            card(Color.EICHEL, Value.ACHT), 
            card(Color.EICHEL, Value.KOENIG),
            card(Color.HERZ, Value.SAU)
        ], None, 2, 0)

[0.01010101 0.92929293 0.06060606]


FigureCanvasNbAgg()

[1.8062188e-04 4.0544182e-01 4.0708482e-04 6.6482770e-04 3.1092277e-04
 1.5561111e-04 9.0298660e-02 8.2208670e-04 3.8172072e-04 2.0914595e-04
 3.2153123e-04 3.2620661e-05 1.4715923e-03 3.2457788e-04 3.1679973e-04
 9.4266092e-05 6.2101352e-01 1.4711650e-04 3.9107932e-04 4.1544883e-04
 2.8779832e-04 1.5081355e-04 6.4754960e-05 1.7648176e-04 5.6564109e-04
 5.9753237e-04 3.1045036e-04 1.8377563e-04 2.3134120e-04 5.0115038e-04
 3.9571812e-04 1.0235918e-03] [1. 1. 0.]
Color.EICHEL Value.ACHT => 0.09029866 (1.0)
Color.EICHEL Value.KOENIG => 0.40544182 (1.0)
Color.HERZ Value.SAU => 0.6210135 (0.0)


In [117]:
predict([
    card(Color.EICHEL, Value.UNTER), 
    card(Color.GRUEN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SIEBEN),
    card(Color.GRUEN, Value.ZEHN),
    ],card(Color.GRUEN, Value.OBER), 0, 0)

[0.01010101 0.68686869 0.         0.29292929 0.01010101]


FigureCanvasNbAgg()

[9.81625817e-06 5.82903810e-03 7.42667107e-05 3.64174023e-02
 1.17296346e-04 2.68332824e-05 2.59283097e-05 3.44696346e-05
 1.78635796e-03 5.38883269e-01 9.97112002e-05 5.36457628e-06
 3.15477559e-03 1.13232323e-04 1.66387217e-05 6.12819131e-05
 2.62365138e-05 3.91452977e-06 3.74842530e-05 1.03645489e-05
 3.11941199e-04 9.05106208e-06 1.20148726e-03 5.53456485e-01
 4.40091981e-06 1.76454669e-05 1.26139221e-05 3.32581258e-05
 2.26214470e-05 3.61688785e-06 2.48612650e-05 8.24118615e-06] [1. 1. 1. 1. 1.]
Color.EICHEL Value.UNTER => 0.036417402 (1.0)
Color.GRUEN Value.KOENIG => 0.53888327 (1.0)
Color.EICHEL Value.KOENIG => 0.005829038 (1.0)
Color.HERZ Value.SIEBEN => 0.5534565 (1.0)
Color.GRUEN Value.ZEHN => 0.0031547756 (1.0)


In [41]:
predict([
    card(Color.HERZ, Value.SIEBEN), 
    card(Color.GRUEN, Value.KOENIG),
    card(Color.HERZ, Value.ACHT),
    card(Color.EICHEL, Value.SAU),
    card(Color.GRUEN, Value.UNTER),
    ],card(Color.SCHELLN, Value.OBER), 0, 0)

[0.1010101  0.01010101 0.86868687 0.01010101 0.01010101]


FigureCanvasNbAgg()

[0.00443341 0.00287482 0.01186269 0.01436252 0.01789274 0.01674011
 0.00073267 0.01019018 0.01441486 0.01178858 0.0144674  0.00262282
 0.01988386 0.01053054 0.00723378 0.00354249 0.00975365 0.00434553
 0.00260457 0.00133964 0.00652008 0.00299097 0.00209025 0.00920635
 0.00844523 0.01307761 0.00444526 0.00520805 0.01260752 0.00672432
 0.00713151 0.00684194] [0. 0. 0. 0. 0.]
Color.HERZ Value.SIEBEN => 0.009206355 (0.0)
Color.GRUEN Value.KOENIG => 0.0117885815 (0.0)
Color.HERZ Value.ACHT => 0.0020902527 (0.0)
Color.EICHEL Value.SAU => 0.0044334056 (0.0)
Color.GRUEN Value.UNTER => 0.002622815 (0.0)


In [395]:
hard_input = generate_input([
    card(Color.EICHEL, Value.UNTER), 
    #card(Color.GRUEN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SIEBEN),
    card(Color.GRUEN, Value.ZEHN),
    ],None, 1, 0)
model.predict_single(hard_input)

[array([3.2768894e-03, 1.9878964e-01, 1.3820220e-02, 4.6424335e-01,
        7.1053780e-03, 2.9274071e-02, 8.1222923e-03, 5.4989127e-04,
        8.0019347e-03, 4.3923180e-03, 4.0565468e-03, 3.2476720e-03,
        2.0850727e-01, 4.3509626e-03, 5.9748022e-03, 6.1062318e-03,
        1.3822509e-03, 2.1725877e-04, 1.3098384e-03, 7.1340741e-04,
        2.0487478e-03, 3.3812402e-03, 1.0878793e-04, 7.3019944e-02,
        3.4260689e-03, 2.7179192e-03, 1.3691070e-03, 6.1381124e-03,
        4.5132274e-03, 4.4453251e-03, 2.2470709e-03, 4.3023235e-04],
       dtype=float32), array([0.5526783], dtype=float32)]

In [ ]:
for sample_input in sample_inputs[0]:
    if np.array_equal(sample_input,hard_input[0]):
        print("test")

In [80]:
compare(model, None)

0.48

In [72]:
match([model, None], True)

[0. 0. 1. 0. 0.]
[1. 1. 0. 1.]
[1. 1. 0.]
[1. 0.]
[1.]


1